In [1]:
import pandas as pd
import numpy as np
from select_feature import make_data
import os

In [25]:
# 성능 평가 함수 생성
def mape(t, y):
    return 100*(np.abs((y-t)/t).sum())/t.shape[0]

In [62]:
data = pd.read_csv("./train_0924.csv")

In [75]:
data

,노출(분),holiday(includeSS),계절,분류,Active Users,new판매단가,new주문량,prime_time,prime_day,top_code,...,판매단가,주문량,예보_최고기온,예보_최저기온,예보_강수확률,예보_강수량,예보_풍속,무이자,일시불,상품군
0,20,1,2,f1,118574,11.686879,4.933022,7,1,0,...,119000,138.798319,25.912281,16.333333,9.279279,0.122807,2.098874,0,0,inner
1,20,1,2,f1,118574,11.686879,5.931487,7,1,0,...,119000,376.714286,25.912281,16.333333,9.279279,0.122807,2.098874,0,0,inner
2,20,1,2,f1,118574,11.686879,6.155001,7,1,0,...,119000,471.067227,25.912281,16.333333,9.279279,0.122807,2.098874,0,0,inner
3,30,1,2,g11,118574,11.141862,4.975834,6,1,0,...,69000,144.869565,25.912281,16.333333,9.279279,0.122807,2.098874,0,0,cloth
4,30,1,2,g11,118574,11.141862,4.915315,6,1,0,...,69000,136.362319,25.912281,16.333333,9.279279,0.122807,2.098874,0,0,cloth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7508,20,1,2,a1,149965,13.381646,4.390192,3,1,9,...,648000,80.655864,28.596491,19.375000,21.351351,0.043860,1.118018,0,0,furniture
7509,20,1,2,i29,149965,12.089539,3.678574,3,1,0,...,178000,39.589888,28.596491,19.375000,21.351351,0.043860,1.118018,0,0,etc
7510,20,1,2,i29,149965,12.089539,3.747652,3,1,0,...,178000,42.421348,28.596491,19.375000,21.351351,0.043860,1.118018,0,0,etc
7511,20,1,2,i29,149965,12.089539,4.888235,3,1,0,...,178000,132.719101,28.596491,19.375000,21.351351,0.043860,1.118018,0,0,etc


#### 모든 feature 넣었을 때

In [83]:
from sklearn.model_selection import cross_val_predict

def feature_selection(data):
    data = data.drop(["주문량","판매단가"], axis=1)
    data = pd.concat([data, pd.get_dummies(data["분류"])], axis=1).drop(["분류"], axis=1)
    for c,x in data.groupby(["상품군"]):
        del x["상품군"]
        # 카테고리 이름, X(표준화한), y 반환
        yield c, StandardScaler().fit_transform(x.drop(["new주문량"], axis=1)), x["new주문량"]

In [84]:
predict_mapes = {}
for c, X, y in feature_selection(data):
    # cross_val_score
    xg = xgboost.XGBRegressor(max_depth = 10, n_estimators = 1000, learning_rate = 0.01, random_state = 1111)
    predicted = cross_val_predict(xg, X,y, cv=3) # 예측값
    error = mape(predicted, y) # mape
    print(c,mape(predicted, y))
    predict_mapes[c]=error

[22:35:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
beauty 6.758525216361524
[22:35:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
bedding 14.860878239038515
[22:35:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:50] WARNING: 

In [85]:
pd.DataFrame([list(predict_mapes.keys()), list(predict_mapes.values())]).T

,0,1
0,beauty,6.75853
1,bedding,14.8609
2,cloth,12.1272
3,elec,28.9809
4,etc,23.6327
5,food,5.70321
6,furniture,27.7996
7,health,10.1322
8,inner,10.602
9,life,18.4936


In [88]:
predict_mapes_copy = predict_mapes.copy()

for k in predict_mapes.keys():
    long = data.loc[data["상품군"]==k,:].shape[0]
    predict_mapes_copy[k]/=long
    
cost = 0
for i in predict_mapes_copy.values():
    cost += i

In [91]:
cost

0.34007440311959247

In [112]:
data.columns

Index(['노출(분)', 'holiday(includeSS)', '계절', '분류', 'Active Users', 'new판매단가',
       'new주문량', 'prime_time', 'prime_day', 'top_code', 'top_cat', 'x1_cat',
       'x2_cat', 'top_real_weather', '실제_최고기온', '실제_최저기온', '실제_강수량', '실제_평균풍속',
       '세일', '판매단가', '주문량', '예보_최고기온', '예보_최저기온', '예보_강수확률', '예보_강수량', '예보_풍속',
       '무이자', '일시불', '상품군'],
      dtype='object')

#### 각 feature을 빼보면서 비교

In [153]:
def feature_selection_1(data):
    data = data.drop(["주문량","판매단가", "top_cat"], axis=1)
    data = pd.concat([data, pd.get_dummies(data["분류"])], axis=1).drop(["분류"], axis=1)
    for c,x in data.groupby(["상품군"]):
        del x["상품군"]
        # 카테고리 이름, X(표준화한), y 반환
        yield c, StandardScaler().fit_transform(x.drop(["new주문량"], axis=1)), x["new주문량"]

In [154]:
predict_mapes_1 = {}
for c, X, y in feature_selection_1(data):
    # cross_val_score
    xg = xgboost.XGBRegressor(max_depth = 10, n_estimators = 1000, learning_rate = 0.01, random_state = 1111)
    predicted = cross_val_predict(xg, X,y, cv=3) # 예측값
    error = mape(predicted, y) # mape
    print(c,mape(predicted, y))
    predict_mapes_1[c]=error

[01:18:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:18:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:18:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
beauty 6.779212541801042
[01:18:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:18:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:18:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
bedding 14.860878239038515
[01:18:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:18:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:18:35] WARNING: 

In [155]:
pd.DataFrame([list(predict_mapes_1.keys()), list(predict_mapes_1.values())]).T

,0,1
0,beauty,6.77921
1,bedding,14.8609
2,cloth,12.1875
3,elec,28.9318
4,etc,23.9576
5,food,5.69944
6,furniture,27.7996
7,health,10.1322
8,inner,10.2118
9,life,18.4936


In [156]:
predict_mapes_copy_1 = predict_mapes_1.copy()

for k in predict_mapes_1.keys():
    long = data.loc[data["상품군"]==k,:].shape[0]
    predict_mapes_copy_1[k]/=long
    
cost = 0
for i in predict_mapes_copy_1.values():
    cost += i

In [157]:
cost

0.33992187047563543